In [29]:
device = 'cpu'

In [1]:
with open('use_for_fine_tune.txt', 'r', encoding='utf-8') as file:
    text = file.read()

print(text[:200])

tanrı ve din dair bir mason sıfatının gereği ahlak yasasına uymak zorundadır ve eğer sanat doğru anlıyorsa asla budala bir ateist veya dinsiz bir liberten olmayacaktır fakat eski zamanlarda masonlar h


In [2]:
# get vocabulary
words = text.lower().split()
print(words[:200])

['tanrı', 've', 'din', 'dair', 'bir', 'mason', 'sıfatının', 'gereği', 'ahlak', 'yasasına', 'uymak', 'zorundadır', 've', 'eğer', 'sanat', 'doğru', 'anlıyorsa', 'asla', 'budala', 'bir', 'ateist', 'veya', 'dinsiz', 'bir', 'liberten', 'olmayacaktır', 'fakat', 'eski', 'zamanlarda', 'masonlar', 'her', 'ülkede', 'ülkenin', 'ya', 'da', 'ulusun', 'dininden', 'olmak', 'bu', 'her', 'ne', 'idiyse', 'şimdi', 'ise', 'özel', 'kanıları', 'kendilerine', 'bırakılarak', 'bütün', 'insanların', 'mütabık', 'oldukları', 'dinden', 'olmaları', 'yeterli', 'görülmektedir', 'bu', 'da', 'onları', 'ayırt', 'eden', 'sıfat', 've', 'bağlantıları', 'ne', 'olursa', 'olsun', 'iyi', 've', 'doğru', 'veya', 'onurlu', 've', 'dürüst', 'insanlar', 'olmaktır', 'böylece', 'masonluk', 'aksi', 'takdirde', 'birbirlerinden', 'ebediyen', 'uzakta', 'kalacak', 'kişilerin', 'arasında', 'bir', 'birlik', 'merkezi', 'hakiki', 'dostluğun', 'bir', 'vesile', 'olur', 'devlet', 'erkanı', 've', 'memurlara', 'dair', 'bir', 'mason', 'nerede', 'yaş

In [3]:
print("Total number of words: {}".format(len(words)))
print("Total number of unique words: {}".format(len(set(words)))) 

Total number of words: 7386835
Total number of unique words: 346505


In [4]:
# Create look-up tables

from collections import Counter
word_counts = Counter(words)
sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True) # descending freq order
idx_to_word = {ii: word for ii, word in enumerate(sorted_vocab)}
word_to_idx = {word: ii for ii, word in idx_to_word.items()}
int_words = [word_to_idx[word] for word in words]

In [5]:
import random
import numpy as np

def subsample_words(int_words, threshold = 1e-5):
    word_counts = Counter(int_words)
    total_n_words = len(int_words)
    
    # calculate the frequency ratio of each word by dividing its count by the total number of words.
    # Total word count / frequency of that particular word = frequency ratio
    freq_ratios = {word: count/total_n_words for word, count in word_counts.items()}
    
    # 1 - sqrt(threshold / frequency ratio)
    p_drop = {word: 1 - np.sqrt(threshold/freq_ratios[word]) for word in word_counts}

    return [word for word in int_words if random.random() < (1 - p_drop[word])]

In [6]:
new_train_words = subsample_words(int_words)
print("New total number of words: {}".format(len(new_train_words)))
print("New total number of unique words: {}".format(len(set(new_train_words)))) 

New total number of words: 3318942
New total number of unique words: 346505


In [7]:
list(idx_to_word.items())[:5]

[(0, 've'), (1, 'bir'), (2, 'bu'), (3, 'ile'), (4, 'da')]

In [8]:
# Load pre-trained word embeddings from JSON file
import json
with open('reduced_word2vec_embeddings.json', 'r', encoding='utf-8') as file:
    pre_trained_embeddings = json.load(file)

In [9]:
list(pre_trained_embeddings.items())[:5]

[('ve',
  [0.8816946744918823,
   -1.1479138135910034,
   -2.8935675621032715,
   1.0134391784667969,
   0.43224236369132996,
   -0.7720584869384766,
   0.24457700550556183,
   -0.3202035427093506,
   -1.2225228548049927,
   0.05931022763252258,
   -4.442621231079102,
   -0.4337739944458008,
   4.712981700897217,
   2.2129993438720703,
   0.2108769565820694,
   1.4924626350402832,
   1.9456349611282349,
   0.8886289596557617,
   -0.656963586807251,
   -0.10049226880073547,
   -0.14904306828975677,
   -0.8109055161476135,
   8.04728889465332,
   1.5026955604553223,
   -0.7928041815757751,
   -1.639419436454773,
   -3.34102463722229,
   -0.7595430016517639,
   -0.8243455290794373,
   1.3045505285263062,
   2.0390782356262207,
   -0.9678201675415039,
   -0.6398859024047852,
   4.949254035949707,
   0.8345519304275513,
   0.3812500536441803,
   -0.06903450936079025,
   -0.23374135792255402,
   4.882972717285156,
   -0.689859926700592]),
 ('kategori',
  [1.7712326049804688,
   -1.6008830070

In [10]:
def get_batches(words, batch_size, max_window_size=5): # 5 - 20

    # calculates the number of full batches that can be created from the given words list
    n_batches = len(words)//batch_size
    
    # truncates the words list to contain only the complete batches
    # discarding any remaining words that cannot form a full batch.
    words = words[:n_batches*batch_size]
    
    
    for i in range(0, len(words), batch_size):
        # creates a batch of center words by extracting a sublist of words
        batch_of_center_words = words[i:i+batch_size]
        
        # To store corresponding center(input) and context(output) words for current batch
        batch_x, batch_y = [], []  

        for ii in range(len(batch_of_center_words)):  # range(batch_size) unless truncated at the end
            x = [batch_of_center_words[ii]]             # single word
            
            y = get_context(words=batch_of_center_words, idx=ii, max_window_size=max_window_size)  # list of context words
            
            # extends the batch_x list by repeating the current center word (x) for each context word in the list (y). 
            # ensures that each center word is associated with its corresponding context words.
            batch_x.extend(x * len(y))
            batch_y.extend(y)
    
        yield batch_x, batch_y       # ex) [1,1,2,2,2,2,3,3,3,3], [0,2,0,1,3,4,1,2,4,5]

In [12]:
# Generate context targets
import random
def get_context(words, idx, max_window_size=5): # 5 - 20
    
    # Generate a random window size R between 1 and max_window_size
    random_window_size = random.randint(1, max_window_size)
    
    # Define the start and end indices for the context window around the target word at index 'idx'
    start = max(0, idx - random_window_size)
    end = min(idx + random_window_size, len(words) - 1)
    
    contexts = words[start:idx] + words[idx + 1:end + 1] # +1 since doesn't include this idx
    return contexts

In [13]:
# Random Example to check context-target words
for_showing_example = [i for i in range(55)]
idx = 23

context_words = get_context(for_showing_example, idx=idx, max_window_size=5)
print(f"Context words for the word at index {idx}: {context_words}")

Context words for the word at index 23: [19, 20, 21, 22, 24, 25, 26, 27]


In [32]:
import torch
from torch import nn
import torch.optim as optim

class SkipGramNeg(nn.Module):
    def __init__(self, n_vocab, n_embed, noise_dist=None, pretrained_embeddings=None):
        super().__init__()
        
        self.n_vocab = n_vocab
        self.n_embed = n_embed
        self.noise_dist = noise_dist
        
        self.in_embed = nn.Embedding(n_vocab, n_embed)
        self.out_embed = nn.Embedding(n_vocab, n_embed)
        
        # Initialize both embedding tables with uniform distribution
        self.in_embed.weight.data.uniform_(-1, 1)
        self.out_embed.weight.data.uniform_(-1, 1)
        
        if pretrained_embeddings is not None:
            # Use a scale factor to control the influence of pre-trained embeddings
            scale_factor = 0.1
            for word, index in word_to_idx.items():
                if word in pretrained_embeddings:
                    pretrained_embedding = torch.FloatTensor(pretrained_embeddings[word])
                    self.in_embed.weight.data[index] = pretrained_embedding * scale_factor
                    self.out_embed.weight.data[index] = pretrained_embedding * scale_factor
                else:
                    # If the word is not in pretrained_embeddings, try variations by removing the last character
                    original_word = word
                    while len(word) > 0:
                        word = word[:-1]
                        if word in pretrained_embeddings:
                            pretrained_embedding = torch.FloatTensor(pretrained_embeddings[word])
                            self.in_embed.weight.data[index] = pretrained_embedding * scale_factor
                            self.out_embed.weight.data[index] = pretrained_embedding * scale_factor
                            break
                    else:
                        print(f"Warning: No matching pre-trained embedding found for '{original_word}'")
        
        
    def forward_input(self, input_words):
        input_vectors = self.in_embed(input_words)
        return input_vectors  # input vector embeddings
    

    def forward_target(self, output_words):
        output_vectors = self.out_embed(output_words)
        return output_vectors  # output vector embeddings
    

    def forward_noise(self, batch_size, n_samples=5):
        """Generate noise vectors with shape (batch_size, n_samples, n_embed)"""
        # If no Noise Distribution specified, sample noise words uniformly from vocabulary
        if self.noise_dist is None:
            noise_dist = torch.ones(self.out_embed.weight.size(0))
        else:
            noise_dist = self.noise_dist
            
        # torch.multinomial:
        # Returns a tensor where each row contains (num_samples) **indices** sampled from 
        # multinomial probability distribution located in the corresponding row of tensor input.
        noise_words = torch.multinomial(input=noise_dist,  # input tensor containing probabilities
                                        num_samples=batch_size * n_samples,  # number of samples to draw
                                        replacement=True)
        noise_words = noise_words.to(device)
        
        # Use the out_embed matrix for embedding noise samples
        noise_vectors = self.out_embed(noise_words).view(batch_size, n_samples, -1)
        
        return noise_vectors

In [33]:
class NegativeSamplingLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self,input_vectors,output_vectors,noise_vectors):
        batch_size, embed_size = input_vectors.shape
    
        input_vectors = input_vectors.view(batch_size, embed_size, 1)   # batch of column vectors
        output_vectors = output_vectors.view(batch_size, 1, embed_size) # batch of row vectors
    
        # log-sigmoid loss for correct pairs
        out_loss = torch.bmm(output_vectors, input_vectors).sigmoid().log().squeeze()
    
        # log-sigmoid loss for incorrect pairs
        noise_loss = torch.bmm(noise_vectors.neg(), input_vectors).sigmoid().log()
        noise_loss = noise_loss.squeeze().sum(1)  # sum the losses over the sample of noise vectors

        return -(out_loss + noise_loss).mean()  # average batch loss

In [34]:
# Get the frequency of each word
freq = Counter(int_words)
freq

Counter({0: 204081,
         1: 128350,
         2: 100408,
         3: 63290,
         4: 50067,
         5: 49969,
         6: 37548,
         7: 33675,
         8: 28155,
         9: 24135,
         10: 23783,
         11: 21677,
         12: 20839,
         13: 20804,
         14: 20668,
         15: 20300,
         16: 19957,
         17: 19492,
         18: 18691,
         19: 18586,
         20: 17897,
         21: 17507,
         22: 17225,
         23: 16985,
         24: 15937,
         25: 15743,
         26: 15291,
         27: 14931,
         28: 14893,
         29: 14275,
         30: 13940,
         31: 13266,
         32: 12681,
         33: 12509,
         34: 11592,
         35: 11022,
         36: 10884,
         37: 10650,
         38: 10648,
         39: 10150,
         40: 10005,
         41: 9794,
         42: 9671,
         43: 9640,
         44: 9302,
         45: 9249,
         46: 9095,
         47: 8757,
         48: 8736,
         49: 8617,
         50: 859

In [35]:
# Calculate the ratio of each word's frequency to the total number of words in int_words.
freq_ratio = {word:cnt/len(word_to_idx) for word, cnt in freq.items()}
freq_ratio

{2398: 0.0011284108454423457,
 0: 0.5889698561348321,
 689: 0.0038152407613165754,
 395: 0.006178843018138267,
 1: 0.37041312535172655,
 5208: 0.000499271294786511,
 19600: 0.00010389460469545894,
 1103: 0.0025079003188987173,
 5125: 0.0005079291785111326,
 23053: 8.369287600467526e-05,
 7075: 0.00035785919395102525,
 2531: 0.0010620337368869137,
 188: 0.010989740407786323,
 2546: 0.0010562618144038325,
 181: 0.011266792686974215,
 128459: 5.771922483081052e-06,
 812: 0.003269794086665416,
 27443: 6.63771085554321e-05,
 32800: 5.194730234772947e-05,
 12: 0.060140546312463024,
 25010: 7.503499228005368e-05,
 180502: 2.885961241540526e-06,
 6915: 0.0003665170776756468,
 164: 0.011996940881083967,
 346: 0.006877245638591074,
 2221: 0.001217875643930102,
 11734: 0.0001991313256662963,
 16: 0.05759512849742428,
 815: 0.003246706396733092,
 705: 0.0037430917302780625,
 18: 0.05394150156563397,
 4: 0.14449142148020952,
 7669: 0.00032611362029407945,
 41147: 3.751749614002684e-05,
 141: 0.0138

In [36]:
# create a distribution where more frequent words have higher probabilities.
freq_ratio = np.array(sorted(freq_ratio.values(), reverse=True))
freq_ratio

array([5.88969856e-01, 3.70413125e-01, 2.89773596e-01, ...,
       2.88596124e-06, 2.88596124e-06, 2.88596124e-06])

In [37]:
# Normalize the sorted frequency ratios to create a probability distribution
# This distribution represents a unigram distribution, 
# where each word's probability is proportional to its frequency in the dataset.
unigram_dist = freq_ratio / freq_ratio.sum() 
noise_dist = torch.from_numpy(unigram_dist**0.75 / np.sum(unigram_dist**0.75))

In [38]:
word_to_idx['din']

689

In [39]:
from torch import optim

embedding_dim = 40 #same as the bottleneck of the encoder

model = SkipGramNeg(len(word_to_idx), embedding_dim, noise_dist, pretrained_embeddings=pre_trained_embeddings)

criterion = NegativeSamplingLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [40]:
x = find_matching_embedding(idx_to_word[int(5)], pre_trained_embeddings, embedding_dim)
x

tensor([ 1.1421, -1.0766, -3.4390,  0.8963,  0.6954, -1.5197,  0.4965, -0.1829,
        -1.6474, -0.2481, -4.9051, -0.4368,  2.9416,  2.1403,  0.4963,  1.7801,
        -4.3500,  0.7556, -0.6901,  4.8769, -0.7101, -1.0796,  2.3750,  2.0848,
        -1.3360, -1.8257,  2.0114, -0.8902, -0.8759,  7.6383,  2.3188, -0.8019,
        -1.4621, -0.0841,  0.6441,  0.8171, -0.2055, -0.4703,  2.7328, -0.8474])

In [44]:
def cosine_similarity(embedding, n_valid_words=16, valid_window=100):

    all_embeddings = embedding.weight  # (n_vocab, n_embed) 
    # sim = (a . b) / |a||b|
    magnitudes = all_embeddings.pow(2).sum(dim=1).sqrt().unsqueeze(0) # (1, n_vocab)

    # Pick validation words from 2 ranges: (0, window): common words & (1000, 1000+window): uncommon words 
    valid_words = random.sample(range(valid_window), n_valid_words//2) + random.sample(range(1000, 1000+valid_window), n_valid_words//2)
    valid_words = torch.LongTensor(np.array(valid_words)).to(device) # (n_valid_words, 1)

    valid_embeddings = embedding(valid_words) # (n_valid_words, n_embed)
    # (n_valid_words, n_embed) * (n_embed, n_vocab) --> (n_valid_words, n_vocab) / 1, n_vocab)
    similarities = torch.mm(valid_embeddings, all_embeddings.t()) / magnitudes  # (n_valid_words, n_vocab)

    return valid_words, similarities

In [47]:
from tqdm import tqdm

def train_skipgram(model,criterion,optimizer,int_words,n_negative_samples=5,batch_size=512,n_epochs=5):
    
    model.to(device)
    step = 0

    for epoch in range(n_epochs):
        for inputs, targets in get_batches(int_words, batch_size=batch_size):
            step += 1
            inputs = torch.LongTensor(inputs).to(device)    # [b*n_context_words]
            targets = torch.LongTensor(targets).to(device)  # [b*n_context_words]

            embedded_input_words = model.forward_input(inputs)
            embedded_target_words = model.forward_target(targets)
            embedded_noise_words = model.forward_noise(batch_size=inputs.shape[0], n_samples=n_negative_samples)

            loss = criterion(embedded_input_words, embedded_target_words, embedded_noise_words)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (step % 1000) == 0:
                print("Epoch: {}/{}".format((epoch+1), n_epochs))
                print("Loss: {:.4f}".format(loss.item()))
                valid_idxs, similarities = cosine_similarity(model.in_embed)
                _, closest_idxs = similarities.topk(6)
                valid_idxs, closest_idxs = valid_idxs.to('cpu'), closest_idxs.to('cpu')
        
                for ii, v_idx in enumerate(valid_idxs):
                    closest_words = [idx_to_word[idx.item()] for idx in closest_idxs[ii]][1:]
                    print(idx_to_word[v_idx.item()] + " | "+ ", ".join(closest_words))

                print("\n...\n")

In [48]:
train_skipgram(model,
               criterion,
               optimizer,
               int_words,
               n_negative_samples=5)

Epoch: 1/5
Loss: 2.6500
terör | yasakları, mektuplarla, baskılarını, meclisinizi, imtiyazlar
telefon | saatleri, girişilmemesi, girişi, oturma, cüzdan
dedi | arar, kendimi, arayıp, düşünmeden, gitme
değil | bize, elbette, gelmez, uyan, böyle
dediği | dediğive, dediğivedat, dediğijlmıra, dediğiveysi, dediğiveli
iki | dört, sayı, ise, sekiz, üç
bütün | zorunlu, türlü, gereği, doğrudan, tüm
sen | başıma, yâr, bilmem, vermem, paşam
inceleme | değerlendirme, belgeleri, incelemeler, hesap, raporlar
beyaz | avcıyım, kollarında, siyah, mavi, tarlaya
belgeleri | kuralları, toplantılar, inceleme, raporlar, öğrencilerin
yapılmış | oluşturulan, sınırlıdır, toplanır, erişebilir, bulunabilirler
noyan | noyan, noyana, noyannın, noyanı, noyanın
gidip | açıp, koyup, girmeye, gidecek, hile
denetim | faiz, harcama, koordinasyon, kazanç, güvenlik
süresi | toplam, değişiklikleri, aralıksız, iptal, üzere

...

Epoch: 1/5
Loss: 3.1338
ile | birlikte, yönetim, görev, kurulu, ve
olduğu | şekilde, olarak, kendi

Epoch: 1/5
Loss: 2.3649
başka | sadece, hangi, tek, açık, hiçbir
sen | sana, bakalım, dedin, bak, kardeşim
ya | oldu, geldi, şimdi, işte, var
şu | anda, olacak, var, tane, ne
en | az, son, içinde, biri, büyük
ancak | ise, üzerine, aynı, ayrıca, halinde
türkiye | avrupa, cumhuriyeti, birliği, büyük, başta
olarak | tarafından, diğer, ile, dışında, ise
teşvik | organize, yetkilileri, kurup, yetkililerinin, kurarak
maddesinde | dayanılarak, maddede, fıkrasında, maddesindeki, hükümlerin
istediği | istediğini, görüşmek, savcılığa, ulaşmaya, söyleyerek
toplantıya | toplantıda, toplantılarına, katılacak, toplantısına, toplantısında
adli | başkanlığının, başkanlığınca, onayına, onayı, kuruluna
tanık | ifadesi, beyanları, müşteki, tanıyıp, anlaşıldığından
çıkar | olduğunda, kararlı, çıkarsa, istekli, olursak
kimsenin | işlerden, olmayacağını, muhatap, kimseden, çıkarmaya

...

Epoch: 1/5
Loss: 2.6772
iki | dört, üç, beş, yedi, yirmi
da | de, ya, oldu, burada, bugün
daha | çok, en, aynı, burada, 

Epoch: 2/5
Loss: 1.7656
dedi | dedim, demiş, bende, dediler, benden
diğer | oldukları, aralarında, irtibatları, bulunduğu, kullandığı
iki | üç, dört, kaç, beş, yaklaşık
ali | durmuş, ahmet, ibrahim, emre, yusuf
çok | önemli, pek, iyi, daha, güzel
ise | olup, kişilerden, ayrıca, kuddisi, olan
isimli | adlı, içeriğinde, doc, incelendiğinde, yazan
iyi | iyidir, biliyorum, hocam, paşam, anladım
yapar | yapamaz, artı, yapacak, emrin, varsa
mahallesi | caddesi, ilçesi, çarşısı, kağıthane, çarşı
komutanlığı | komutanlığının, jandarma, komutanlığında, asayiş, komutanlığına
toplantıya | toplantıda, toplantıyı, konferansa, toplantıdan, toplantısından
bakanı | direktörü, direktörümüzün, direktörümüz, direktörümüzle, direktörüyle
tabii | açımızdan, yaşıyoruz, cumhuriyette, hiçbirisi, bizimki
sayfa | çıktısı, sayfalık, doküman, klasör, word
gerekir | vermeniz, gerekirdi, gerekiyormuş, fayda, unutmamak

...

Epoch: 2/5
Loss: 2.5620
onun | onların, doğru, fakat, onlar, onlara
devlet | türk, temsilen,

Epoch: 3/5
Loss: 2.2715
ne | nasıl, benim, nerede, sen, kimin
olan | oldukları, olup, ise, olduğu, olduğundan
karşı | kalarak, karşıya, rağmen, diktatörlüğüne, müdahalesine
da | de, ya, patlattığı, anlaşılmaktadır, âşûb
mehmet | osman, murat, halil, özcan, erdal
ın | in, un, abdullah, yıldırım, çağlar
ki | eğer, şöyle, çünkü, öyle, kim
onun | fakat, onların, onlar, allahın, ona
msword | xls, word, doc, excel, ppt
demir | bulut, esen, ipek, kaya, topuz
gidip | eve, gittim, giderken, gelip, gelirken
mahallesi | caddesi, selimiye, çarşısı, ilçesi, nallıhan
buradan | doğalgazı, bakın, birileri, bugün, sesleniyorum
tan | ten, dan, aziz, den, yu
birer | ikişer, üçer, birinde, altışar, beşyüz
halil | nusret, ziya, hüseyin, ibrahim, yorulmaz

...

Epoch: 3/5
Loss: 2.4037
var | hani, burda, mı, yok, varmış
işte | şimdi, artık, bizim, zaten, yani
dediği | tamam, anladım, abi, hı, hocam
şimdi | işte, yarın, valla, dedim, bak
ancak | bulunmadığını, isede, söylendiğini, geçmediğini, kendisinin
birl

Epoch: 3/5
Loss: 2.2539
ın | in, un, nın, yıldırım, nu
onun | fakat, onlar, onların, ona, allahın
iki | üç, dört, beş, yedi, altı
saat | günü, tape, aradığı, görüşme, sıralarında
şey | şeyi, öyle, şeye, yani, bizde
türkiye | cumhuriyeti, ilelebet, yararınadır, iktidarıyla, devleti
daha | çok, biraz, pek, bundan, burada
olduğunu | olduklarını, olmadığını, bulunduğunu, olabileceğini, olacağını
halil | hasan, ercan, ziya, ibrahim, nusret
heyet | vekile, temsiliye, kabine, riyâseti, riyâset
planı | planının, eylem, çalışması, planında, planıdır
kimsenin | kimseyi, bilmeyerek, başkasının, kuşkusu, kimseden
msword | xls, doc, word, pdf, ppt
abd | ab, amerikan, rusya, azerbaycan, israil
kararları | kararlarını, kararlar, içtihadı, kararlarının, taslağını
şüpheliler | şüpheli, şüphelilerden, alperen, sanıklar, düzalan

...

Epoch: 3/5
Loss: 2.0087
yer | alan, aldığı, şeklinde, tamamının, tamamı
çok | daha, pek, iyi, biraz, önemli
nin | yi, yle, deki, ye, nın
de | hem, da, yi, yine, bugün
bulun

Epoch: 4/5
Loss: 1.8928
bana | benden, sana, dedim, dedi, bende
hakkında | hakkındaki, duruşmadaki, haklarında, mercilerin, kapsamındaki
bulunan | üzerinde, muhtelif, bulunduğu, içerisindeki, bulanan
gibi | 𐰖𐰾𐰴𐰀, cûdda, vardır, 𐰴𐰃𐱁𐰑𐰢𐰕, sõzle
var | burda, hani, bide, varya, vardı
ama | yani, zaten, artık, şey, şimdi
ün | kemal, yalçın, raif, gürüz, paşayı
devlet | devletin, türk, devleti, hukuk, millî
yıldır | dönem, öncesinden, aydır, dönemden, senedir
komutanlığı | komutanlığının, jandarma, komutanlığında, komutanlıkları, karargahında
ederim | ediyorum, istirham, teşekkür, dilerim, ederiz
gösteren | faaliyet, kuruluşlarının, kuruluşlarından, kuruluşlannm, sendikalarının
yapı | sektörler, alanlarının, aplikasyon, çerçevesinin, alanları
söylüyorum | söyledik, şeylerini, konuşuyorum, anlatacağım, inanın
başladı | çıktık, başladım, girdi, girdik, içeri
ağa | kethüdâ, ağası, ağaya, kethüdâsı, gâzi

...

Epoch: 4/5
Loss: 2.1641
gün | günde, sene, hafta, ay, ertesi
ben | dedim, benim, sana, 

Epoch: 5/5
Loss: 2.1195
bunu | bunları, siz, nasıl, sizin, biz
ile | arasındaki, la, arasında, sıralarındaki, hakkında
ilgili | alakalı, konu, birlikte, yaptıklan, gündemdeki
olarak | selamlaşmadan, taşımamakla, gurubunda, хвалу, koordinasyondan
şimdi | işte, bak, valla, zaten, yokum
yok | mı, olmaz, mi, bişey, mu
madde | saklıdır, md, hükümleri, geçici, tabidir
türk | düşünceleridir, cumhuriyeti, federe, ulusunun, ulusuna
buradan | burada, aynısını, bahsediyorum, bakın, kandil
bilmiyorum | bilemiyorum, söylemedim, haberim, soruyor, demişler
emir | talimat, şikayet, emirleri, emri, savcıya
uzman | sınıfında, unvanlı, enstitü, elemanları, sendikasının
biliyorum | söylüyor, oluyorum, bilmiyorum, bilmiyor, görüşüyorum
tanık | ifadesi, beyanları, dinlenen, müşteki, beyanlarını
olduğundan | olup, olduklarından, olmadığından, bulunduğundan, olmuştur
teklif | havale, vekalet, sözleşmeyi, talebinin, teklifin

...

Epoch: 5/5
Loss: 2.0191
de | da, hem, başkasın, dinlemede, yaptıği
türk | düşünc

Epoch: 5/5
Loss: 2.0365
olduğu | oldukları, bulunduğu, olmadığı, olan, olduğunu
onun | ona, onların, fakat, allahın, çokları
ın | in, yıldırım, nın, nu, akın
bana | benden, sana, ben, beni, dedi
ama | zaten, yani, aslında, bunu, birileri
genel | yönetim, kurulu, başkanlığı, sekreter, denetleme
şekilde | sürece, engellenmesini, yatırımlardan, etmemesidir, çalışmalıdır
olduğunu | olduklarını, olmadığını, olabileceğini, bulunduğunu, olacağını
olduğundan | olup, olmayıp, olmuştur, olduklarından, dahi
onunla | onlarla, benimle, bizimle, seninle, onu
gazetesi | gazetesinin, gazetesinde, dergisi, dergisinde, yazınızın
mahmut | ismail, öztürk, ersan, solmaz, hamdi
yapı | yapıların, alanları, alanlarının, üretilecek, sektörler
biliyorum | söylüyor, bilmiyor, yapmayın, söylüyorsun, bilmiyorum
dikkate | gözönüne, alınacağı, kriterler, kapasitelerini, alınmalıdır
dünyanın | ülkesinin, mazlumların, ülkenin, ülkeyiz, üzerindedir

...

Epoch: 5/5
Loss: 2.3201
zaman | vakit, şeyi, halde, işittikleri, 

In [55]:
import torch.nn.functional as F

def most_similar_words(model, word, word_to_idx, idx_to_word, top_n=10):
    """
    Find the most similar words to the given word using cosine similarity.
    Prints each similar word and its similarity value on a separate line.
    """
    if word not in word_to_idx:
        print(f"Word '{word}' not in the vocabulary.")
        return

    # Get the embedding for the given word
    word_idx = word_to_idx[word]
    word_embedding = model.in_embed.weight.data[word_idx]

    # Calculate cosine similarity with all other words
    all_embeddings = model.in_embed.weight.data
    similarities = F.cosine_similarity(word_embedding, all_embeddings)

    # Get indices and values of top similar words
    _, top_indices = similarities.topk(top_n + 1)  # +1 to exclude the word itself

    for idx in top_indices:
        if idx != word_idx:
            similar_word = idx_to_word[idx.item()]
            similarity_value = similarities[idx].item()
            print(f"{similar_word}: {similarity_value:.4f}")


In [57]:
similar_words = most_similar_words(model, 'din', word_to_idx, idx_to_word, top_n=10)
print(similar_words)

ahlâk: 0.8415
felsefe: 0.8329
mezhep: 0.8265
ayrımcılığını: 0.8235
dinin: 0.8025
edebin: 0.7947
varoluş: 0.7946
inançları: 0.7920
edebinden: 0.7879
masonluk: 0.7830
None


In [51]:
# Save the entire model (including weights)
torch.save(model.state_dict(), 'fine_tuned_model.pth')

# Save the learned word embeddings
torch.save(model.in_embed.weight.data, 'fine_tuned_model_word_embeddings.pth')

In [ ]:
# Load the model
loaded_model = SkipGramNeg(len(word_to_idx), embedding_dim, noise_dist)
loaded_model.load_state_dict(torch.load('fine_tuned_model.pth'))
loaded_model.eval()  # Set the model to evaluation mode

In [ ]:
# Load the word embeddings
loaded_embeddings = torch.load('fine_tuned_model_word_embeddings.pth')

In [65]:
similar_words = most_similar_words(model, 'allah', word_to_idx, idx_to_word, top_n=10)
print(similar_words)

alîmdir: 0.8528
gafurdur: 0.8527
rahîmdir: 0.8423
hikmetini: 0.8394
buyuruyor: 0.8279
hakîmdir: 0.8274
allaha: 0.8263
haberiniz: 0.8241
hidayet: 0.8218
kasem: 0.8191
None


In [59]:
similar_words = most_similar_words(model, 'din', word_to_idx, idx_to_word, top_n=10)
print(similar_words)

ahlâk: 0.8415
felsefe: 0.8329
mezhep: 0.8265
ayrımcılığını: 0.8235
dinin: 0.8025
edebin: 0.7947
varoluş: 0.7946
inançları: 0.7920
edebinden: 0.7879
masonluk: 0.7830
None


In [60]:
similar_words = most_similar_words(model, 'felsefe', word_to_idx, idx_to_word, top_n=10)
print(similar_words)

metafizik: 0.8884
teoloji: 0.8755
marksizmin: 0.8620
kavramdır: 0.8578
yapıtında: 0.8526
teolojistlere: 0.8511
ahlak: 0.8505
yazınsal: 0.8498
ahlâk: 0.8476
düşünceyse: 0.8473
None


In [61]:
similar_words = most_similar_words(model, 'mantık', word_to_idx, idx_to_word, top_n=10)
print(similar_words)

fikirler: 0.8339
tutarlılık: 0.8231
arzolunacaklar: 0.8192
metafizik: 0.8153
anlamı: 0.8145
felsefe: 0.8144
soyutlaştırarak: 0.8108
sıfatlar: 0.8073
eşanlamlı: 0.8073
teolojik: 0.8062
None


In [62]:
import torch.nn.functional as F

def word_arithmetic(model, word_to_idx, idx_to_word, list1, list2, top_n=10):
    """
    Perform vector arithmetic: (word1 + word2 + ...) - (word3 + word4 + ...)
    Print the most similar words to the resulting vector.
    """
    # Check if all words are in the vocabulary
    all_words = list1 + list2
    if any(word not in word_to_idx for word in all_words):
        print("One or more words not in the vocabulary.")
        return

    # Get the embeddings for the given words in list1 and list2
    embeddings_list1 = torch.stack([model.in_embed.weight.data[word_to_idx[word]] for word in list1])
    embeddings_list2 = torch.stack([model.in_embed.weight.data[word_to_idx[word]] for word in list2])

    # Calculate the resulting vector: (word1 + word2 + ...) - (word3 + word4 + ...)
    result_embedding = embeddings_list1.sum(dim=0) - embeddings_list2.sum(dim=0)

    # Calculate cosine similarity with all other words
    all_embeddings = model.in_embed.weight.data
    similarities = F.cosine_similarity(result_embedding, all_embeddings)

    # Get indices and values of top similar words
    _, top_indices = similarities.topk(top_n)

    print(f"Words similar to {' + '.join(list1)} - {' + '.join(list2)}:")
    for idx in top_indices:
        similar_word = idx_to_word[idx.item()]
        similarity_value = similarities[idx].item()
        print(f"{similar_word}: {similarity_value:.4f}")

In [63]:
word_arithmetic(model, word_to_idx, idx_to_word, ['kral', 'kadın'], ['erkek'], top_n=10)

Words similar to kral + kadın - erkek:
hıristiyanlarda: 0.7622
hattuşa: 0.7535
fethedilirken: 0.7511
kaşgar: 0.7454
dikilmiştir: 0.7430
yunanlar: 0.7428
gelentrafik: 0.7345
medler: 0.7343
mölsün: 0.7295
ahalisinde: 0.7283


In [64]:
word_arithmetic(model, word_to_idx, idx_to_word, ['geliyor', 'gitmek'], ['gelmek'], top_n=10)

Words similar to geliyor + gitmek - gelmek:
geliyor: 0.7913
dönmüş: 0.7608
yatıyor: 0.7451
kesmiş: 0.7397
ablamın: 0.7369
gitme: 0.7360
gitmiş: 0.7353
giderken: 0.7329
gelme: 0.7264
takmış: 0.7190
